# Exercise 9

Reinforcement Learning Lecture WS 2018/2019

Instructor: Georg Martius georg.martius@tuebingen.mpg.de

Due-date: 21.12.2018


This time we will use Open AI gym again, see https://gym.openai.com/ For the code and installation instructions see https://github.com/openai/gym

This time we don't need additional environments, but you can also install all of them.

The main task for this exercise is to implement Q-learning with neural network function approximation, so a simple version of DQN.



# Initialization

In [ ]:
import gym
from gym import spaces
import numpy as np
import itertools
import time

import memory

In [ ]:
import pylab as plt
%matplotlib inline

# Create Gym Environment

In [ ]:
#env = gym.make('MountainCar-v0')
env = gym.make('CartPole-v0')
#env = gym.make('Acrobot-v1')
ac_space = env.action_space
o_space = env.observation_space
print(ac_space)
print(o_space)
print(zip(env.observation_space.low, env.observation_space.high))

# Q-function

In [ ]:
import sonnet as snt
import tensorflow as tf

class QFunction:
    def __init__(self, o_space, a_space, gamma=0.99, scope='Q'):
        self._scope = scope
        self._o_space = o_space
        self._a_space = a_space
        self._action_n = self._a_space.n
        self._gamma = gamma
        self._sess = tf.get_default_session() or tf.InteractiveSession()
    
        with tf.variable_scope(self._scope, reuse=tf.AUTO_REUSE):
            self._build_graph()
    
    def _build_graph(self):
        self.state = tf.placeholder(dtype=tf.float32, shape=(None, self._o_space.shape[0]), name="state")
        self.action = tf.placeholder(dtype=tf.float32, shape=(None,1), name="action")        
        # we use hardcoded one discrete action input here        
        
        # create network here, look at the code from last week from value fitting. 
        # Use also two hidden layers with 100 relu units
        
        self.output = ...
    
    def Q(self, state, action):
        _state = np.asarray(state).reshape(-1, self._o_space.shape[0])
        _action = np.asarray(action).reshape(-1, 1)
        inp = {self.state: _state, self.action: _action}
        return self._sess.run([self.output], feed_dict=inp)
    
    def V(self, state):
        # compute value for given state 
        return ....

# Deep Q-learning

In [ ]:
class DQNAgent(object):
    """
    Agent implementing Q-learning with NN function approximation.
    """
    def __init__(self, o_space, a_space, scope='DQNAgent', **userconfig):
        
        if not isinstance(o_space, spaces.box.Box):
            raise UnsupportedSpace('Observation space {} incompatible with {}. (Require: Box)'.format(observation_space, self))
        if not isinstance(a_space, spaces.discrete.Discrete):
            raise UnsupportedSpace('Action space {} incompatible with {}. (Reqire Discrete.)'.format(action_space, self))
        
        self._o_space = o_space
        self._a_space = a_space
        self._action_n = a_space.n
        self._config = {
            "eps": 0.05,            # Epsilon in epsilon greedy policies
            "discount": 0.95,
            "buffer_size": int(1e5),
            "batch_size": 50,
            "learning_rate": 1e-4}
        self._config.update(userconfig)
        self._scope = scope
        self._sess = tf.get_default_session() or tf.InteractiveSession()
        
        # Create Q Network and other stuff you need. E.g. 
        with tf.variable_scope(self._scope, reuse=tf.AUTO_REUSE):            
            # define Q and target variable, loss optimizer and train_operation as in value fitting                    
            self._Q = ... 
                        
            self._train_op = ...
    
        self._sess.run(tf.global_variables_initializer())
        
            
    def _vars(self, scope=''):
        return tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=self._scope + '/' + scope)
    
    def _global_vars(self, scope=''):
        return tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=self._scope + '/' + scope)
            
    def act(self, observation, eps=None):
        if eps is None:
            eps = self._config["eps"]
        # epsilon greedy.
        if np.random.random() > eps:
            action = np.argmax([self._Q.Q(observation, a) for a in range(self._action_n)])     
        else: 
            action = self._a_space.sample()        
        return action
    
    def store_transition(self, ob, a, reward, ob_new, done):
        # Fill this part
        
    def train(self, iter_fit=10):
        losses = []
        for i in range(iter_fit):
            # Fill this part
            
        return losses

In [ ]:
fps = 100 # env.metadata.get('video.frames_per_second')
max_steps = env.spec.tags['wrapper_config.TimeLimit.max_episode_steps']

In [ ]:
q_agent = DQNAgent(o_space, ac_space, discount=0.95, eps=0.05)

Test what the q-function outputs, so see if it does not crash

In [ ]:
ob = env.reset()
print(ob)
q_agent._Q.Q(ob, 1)

## Running some episodes and train

In [ ]:
losses = []

In [ ]:
max_episodes=300
#mode="random"
show=False
mode="Q"
for i in range(max_episodes):
    # print("Starting a new episode")    
    total_reward = 0
    ob = env.reset()    
    for t in range(max_steps):
        done = False
        if mode == "random":
            a = ac_space.sample()                        
        elif mode == "Q":
            a = q_agent.act(ob)
        else:
            raise ValueError("no implemented")
        (ob_new, reward, done, _info) = env.step(a)
        total_reward+= reward
        if mode == "Q":
            q_agent.store_transition(ob, a, reward, ob_new, done)            
            losses.extend(q_agent.train(1))
        ob=ob_new        
        if show:
            time.sleep(1.0/fps)
            env.render(mode='human')        
        if done: break    
    
    if ((i-1)%20==0):
        print("Done after {} rollouts. Reward: {}".format(t+1, total_reward))

## Analysis

Print fitting loss, the learning progress etc. 

Do this for several runs of the cart-pendulum

# Bonus: Also try on another environment, e.g. 'MountainCar-v0' or 'Acrobot-v1'